In [1]:
import pandas as pd 
import numpy as np
import os
import statsmodels.api as sm 
import statsmodels.formula.api as smf 

/opt/conda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
ed = pd.read_csv('ed.csv')

In [3]:
ed.head()

,year,stateFIPS,countyFIPS,PPCSTOT,TOTALEXP,TCURSPND
0,2010,8,8001.0,57366,840454,699730
1,2010,8,8003.0,19347,60756,27335
2,2010,8,8005.0,70899,1211781,994837
3,2010,8,8007.0,8411,13701,12904
4,2010,8,8009.0,65738,12341,10428


In [4]:
full = pd.read_csv('full_20102014.csv')

In [5]:
full.head()

,Unnamed: 0,countyFIPS,year,proprietors,population,stateFIPS,quarter,Emp,emp_growth,str,...,KPost2012,logy,qtryr,pop10,percap_emp,tt_CO,tt_KS,tt_MO,tt_NE,tt_OK
0,1536,8001,2010,40124,443681,8,1,128286,-0.031131,2.9,...,0,11.762017,2010.1,44.3681,0.289140,7,0,0,0,0
1,1537,8001,2010,40124,443681,8,2,133394,0.039817,2.9,...,0,11.801062,2010.2,44.3681,0.300653,7,0,0,0,0
2,1538,8001,2010,40124,443681,8,3,135286,0.014184,2.9,...,0,11.815146,2010.3,44.3681,0.304917,7,0,0,0,0
3,1539,8001,2010,40124,443681,8,4,134766,-0.003844,2.9,...,0,11.811295,2010.4,44.3681,0.303745,7,0,0,0,0
4,1540,8003,2010,2188,15927,8,1,5281,-0.075779,2.9,...,0,8.571871,2010.1,1.5927,0.331575,7,0,0,0,0


In [6]:
full.columns

Index(['Unnamed: 0', 'countyFIPS', 'year', 'proprietors', 'population',
       'stateFIPS', 'quarter', 'Emp', 'emp_growth', 'str', 'imtr', 'cmtr',
       'KPost2012', 'logy', 'qtryr', 'pop10', 'percap_emp', 'tt_CO', 'tt_KS',
       'tt_MO', 'tt_NE', 'tt_OK'],
      dtype='object')

In [7]:
educ = ed.merge(full, on = ['year', 'stateFIPS', 'countyFIPS'])

In [8]:
educ = educ.filter(['year', 'stateFIPS', 'countyFIPS', 'PPCSTOT', 'TOTALEXP', 'TCURSPND',
                    'population', 'pop10', 'quarter', 'KPost2012', 'str', 'imtr', 'cmtr',
                    'tt_CO', 'tt_KS', 'tt_NE', 'tt_OK', 'tt_MO'])

In [9]:
educ = educ[educ['quarter'] == 1]

In [10]:
educ.head()

,year,stateFIPS,countyFIPS,PPCSTOT,TOTALEXP,TCURSPND,population,pop10,quarter,KPost2012,str,imtr,cmtr,tt_CO,tt_KS,tt_NE,tt_OK,tt_MO
0,2010,8,8001,57366,840454,699730,443681,44.3681,1,0,2.9,4.63,4.63,7,0,0,0,0
4,2010,8,8003,19347,60756,27335,15927,1.5927,1,0,2.9,4.63,4.63,7,0,0,0,0
8,2010,8,8005,70899,1211781,994837,574785,57.4785,1,0,2.9,4.63,4.63,7,0,0,0,0
12,2010,8,8007,8411,13701,12904,12055,1.2055,1,0,2.9,4.63,4.63,7,0,0,0,0
16,2010,8,8009,65738,12341,10428,3790,0.3790,1,0,2.9,4.63,4.63,7,0,0,0,0


In [11]:
#TotalEXP
sp1 = """np.log(TOTALEXP) ~ KPost2012 + pop10 + str + imtr + cmtr + +C(countyFIPS) + tt_CO +
tt_KS + tt_MO + tt_NE + tt_OK"""
TotalEXPlog = smf.ols(formula=sp1, data=educ).fit(cov_type ='cluster', cov_kwds=
                                             {'groups': educ.stateFIPS})
TotalEXPlog.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       np.log(TOTALEXP)   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                 8.542e+16
Date:                Tue, 01 May 2018   Prob (F-statistic):           2.75e-34
Time:                        21:04:50   Log-Likelihood:                 1932.1
No. Observations:                2260   AIC:                            -2942.
Df Residuals:                    1799   BIC:                            -303.9
Df Model:                         460                                         
Covariance Type:              cluster                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   17.1529      0.763     22.495      0.000      15.658      18.647
C(countyFIPS)[T.8003.0]     -1.7444      0.276     -6.318      0.000      -2.286      -1.203
C(countyFIPS)[T.8005.0]     -0.0168      0.084     -0.200      0.841      -0.181       0.148
C(countyFIPS)[T.8007.0]     -2.7297      0.279     -9.798      0.000      -3.276      -2.184
C(countyFIPS)[T.8009.0]     -3.0529      0.284    -10.757      0.000      -3.609      -2.497
C(countyFIPS)[T.8011.0]     -3.2474      0.282    -11.500      0.000      -3.801      -2.694
C(countyFIPS)[T.8013.0]      0.2593      0.097      2.675      0.007       0.069       0.449
C(countyFIPS)[T.8015.0]     -1.9390      0.275     -7.054      0.000      -2.478      -1.400
C(countyFIPS)[T.8017.0]     -3.7065      0.285    -13.006      0.000      -4.265      -3.148
C(countyFIPS)[T.8019.0]     -2.8567      0.280    -10.185      0.000      -3.406      -2.307
C(countyFIPS)[T.8021.0]     -2.3715      0.281     -8.440      0.000      -2.922      -1.821
C(countyFIPS)[T.8023.0]     -3.1204      0.284    -10.991      0.000      -3.677      -2.564
C(countyFIPS)[T.8025.0]     -3.8540      0.283    -13.635      0.000      -4.408      -3.300
C(countyFIPS)[T.8027.0]     -3.8839      0.283    -13.701      0.000      -4.439      -3.328
C(countyFIPS)[T.8029.0]     -1.5816      0.267     -5.918      0.000      -2.105      -1.058
C(countyFIPS)[T.8031.0]     -0.2075      0.107     -1.947      0.052      -0.416       0.001
C(countyFIPS)[T.8033.0]     -4.1342      0.285    -14.512      0.000      -4.693      -3.576
C(countyFIPS)[T.8035.0]      0.1228      0.100      1.227      0.220      -0.073       0.319
C(countyFIPS)[T.8037.0]     -1.0851      0.254     -4.276      0.000      -1.582      -0.588
C(countyFIPS)[T.8039.0]     -1.7792      0.272     -6.552      0.000      -2.311      -1.247
C(countyFIPS)[T.8041.0]     -0.3244      0.114     -2.842      0.004      -0.548      -0.101
C(countyFIPS)[T.8043.0]     -1.5904      0.257     -6.187      0.000      -2.094      -1.087
C(countyFIPS)[T.8045.0]     -0.7494      0.251     -2.985      0.003      -1.241      -0.257
C(countyFIPS)[T.8047.0]     -3.7155      0.283    -13.143      0.000      -4.270      -3.161
C(countyFIPS)[T.8049.0]     -2.3434      0.277     -8.455      0.000      -2.887      -1.800
C(countyFIPS)[T.8051.0]     -2.1230      0.276     -7.679      0.000      -2.665      -1.581
C(countyFIPS)[T.8053.0]     -4.9508      0.286    -17.333      0.000      -5.511      -4.391
C(countyFIPS)[T.8055.0]     -3.1740      0.282    -11.252      0.000      -3.727      -2.621
C(countyFIPS)[T.8057.0]     -4.2571      0.285    -14.923      0.000      -4.816      -3.698
C(countyFIPS)[T.8059.0]     -0.2275      0.053     -4.305      0.000      -0.331      -0.124
C(countyFIPS)[

In [12]:
#TotalEXP
sp1 = """TOTALEXP ~ KPost2012 + pop10 + str + imtr + cmtr + +C(countyFIPS) + tt_CO +
tt_KS + tt_MO + tt_NE + tt_OK"""
TotalEXP = smf.ols(formula=sp1, data=educ).fit(cov_type ='cluster', cov_kwds=
                                             {'groups': educ.stateFIPS})
TotalEXP.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               TOTALEXP   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 2.512e+16
Date:                Tue, 01 May 2018   Prob (F-statistic):           3.18e-33
Time:                        21:04:52   Log-Likelihood:                -23948.
No. Observations:                2260   AIC:                         4.882e+04
Df Residuals:                    1799   BIC:                         5.146e+04
Df Model:                         460                                         
Covariance Type:              cluster                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                -1.997e+05   3.95e+05     -0.506      0.613   -9.73e+05    5.74e+05
C(countyFIPS)[T.8003.0]   8.976e+04   1.38e+05      0.651      0.515   -1.81e+05     3.6e+05
C(countyFIPS)[T.8005.0]   1.212e+05   4.19e+04      2.894      0.004    3.91e+04    2.03e+05
C(countyFIPS)[T.8007.0]   7.125e+04   1.39e+05      0.512      0.609   -2.02e+05    3.44e+05
C(countyFIPS)[T.8009.0]   8.349e+04   1.42e+05      0.589      0.556   -1.94e+05    3.61e+05
C(countyFIPS)[T.8011.0]   7.726e+04   1.41e+05      0.548      0.584   -1.99e+05    3.54e+05
C(countyFIPS)[T.8013.0]   1.535e+05   4.84e+04      3.170      0.002    5.86e+04    2.48e+05
C(countyFIPS)[T.8015.0]   7.808e+04   1.37e+05      0.569      0.570   -1.91e+05    3.47e+05
C(countyFIPS)[T.8017.0]   8.228e+04   1.42e+05      0.578      0.563   -1.97e+05    3.61e+05
C(countyFIPS)[T.8019.0]   7.531e+04    1.4e+05      0.537      0.591   -1.99e+05     3.5e+05
C(countyFIPS)[T.8021.0]   8.527e+04    1.4e+05      0.607      0.544    -1.9e+05     3.6e+05
C(countyFIPS)[T.8023.0]   8.378e+04   1.42e+05      0.591      0.555   -1.94e+05    3.62e+05
C(countyFIPS)[T.8025.0]   7.432e+04   1.41e+05      0.526      0.599   -2.02e+05    3.51e+05
C(countyFIPS)[T.8027.0]   7.678e+04   1.42e+05      0.542      0.588   -2.01e+05    3.54e+05
C(countyFIPS)[T.8029.0]   6.842e+04   1.34e+05      0.512      0.608   -1.93e+05     3.3e+05
C(countyFIPS)[T.8031.0]  -4.178e+04   5.32e+04     -0.785      0.433   -1.46e+05    6.26e+04
C(countyFIPS)[T.8033.0]   8.018e+04   1.42e+05      0.563      0.573   -1.99e+05    3.59e+05
C(countyFIPS)[T.8035.0]   7.033e+04      5e+04      1.406      0.160   -2.77e+04    1.68e+05
C(countyFIPS)[T.8037.0]   6.127e+04   1.27e+05      0.483      0.629   -1.87e+05     3.1e+05
C(countyFIPS)[T.8039.0]   7.335e+04   1.36e+05      0.541      0.589   -1.93e+05    3.39e+05
C(countyFIPS)[T.8041.0]  -1.549e+05    5.7e+04     -2.717      0.007   -2.67e+05   -4.32e+04
C(countyFIPS)[T.8043.0]   3.819e+04   1.28e+05      0.297      0.766   -2.13e+05     2.9e+05
C(countyFIPS)[T.8045.0]   8.693e+04   1.25e+05      0.693      0.488   -1.59e+05    3.33e+05
C(countyFIPS)[T.8047.0]   7.512e+04   1.41e+05      0.532      0.595   -2.02e+05    3.52e+05
C(countyFIPS)[T.8049.0]   7.352e+04   1.38e+05      0.531      0.595   -1.98e+05    3.45e+05
C(countyFIPS)[T.8051.0]   7.912e+04   1.38e+05      0.573      0.567   -1.92e+05     3.5e+05
C(countyFIPS)[T.8053.0]   8.066e+04   1.43e+05      0.565      0.572   -1.99e+05     3.6e+05
C(countyFIPS)[T.8055.0]   7.694e+04   1.41e+05      0.546      0.585   -1.99e+05    3.53e+05
C(countyFIPS)[T.8057.0]   8.106e+04   1.43e+05      0.569      0.569   -1.98e+05     3.6e+05
C(countyFIPS)[T.8059.0]  -1.445e+05   2.64e+04     -5.475      0.000   -1.96e+05   -9.28e+04
C(countyFIPS)[

In [13]:
#PPCSTOT
sp2 = """np.log(PPCSTOT) ~ KPost2012 + pop10 + str + imtr + cmtr + +C(countyFIPS) + tt_CO +
tt_KS + tt_MO + tt_NE + tt_OK"""
PPCSTOTlog = smf.ols(formula=sp2, data=educ).fit(cov_type ='cluster', cov_kwds=
                                             {'groups': educ.stateFIPS})
PPCSTOTlog.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        np.log(PPCSTOT)   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                -1.862e+18
Date:                Tue, 01 May 2018   Prob (F-statistic):               1.00
Time:                        21:04:53   Log-Likelihood:                 3178.5
No. Observations:                2260   AIC:                            -5435.
Df Residuals:                    1799   BIC:                            -2797.
Df Model:                         460                                         
Covariance Type:              cluster                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   13.0379      0.528     24.687      0.000      12.003      14.073
C(countyFIPS)[T.8003.0]     -1.5057      0.195     -7.707      0.000      -1.889      -1.123
C(countyFIPS)[T.8005.0]      0.3212      0.059      5.414      0.000       0.205       0.437
C(countyFIPS)[T.8007.0]     -2.2335      0.197    -11.329      0.000      -2.620      -1.847
C(countyFIPS)[T.8009.0]     -0.2081      0.201     -1.036      0.300      -0.602       0.186
C(countyFIPS)[T.8011.0]     -1.5346      0.200     -7.680      0.000      -1.926      -1.143
C(countyFIPS)[T.8013.0]     -1.3031      0.069    -19.002      0.000      -1.437      -1.169
C(countyFIPS)[T.8015.0]     -1.4493      0.195     -7.451      0.000      -1.830      -1.068
C(countyFIPS)[T.8017.0]     -0.9784      0.202     -4.852      0.000      -1.374      -0.583
C(countyFIPS)[T.8019.0]     -2.1168      0.198    -10.666      0.000      -2.506      -1.728
C(countyFIPS)[T.8021.0]     -1.0028      0.199     -5.044      0.000      -1.393      -0.613
C(countyFIPS)[T.8023.0]     -1.2956      0.201     -6.449      0.000      -1.689      -0.902
C(countyFIPS)[T.8025.0]     -2.2489      0.200    -11.245      0.000      -2.641      -1.857
C(countyFIPS)[T.8027.0]     -2.2076      0.201    -11.006      0.000      -2.601      -1.814
C(countyFIPS)[T.8029.0]     -2.3765      0.189    -12.566      0.000      -2.747      -2.006
C(countyFIPS)[T.8031.0]     -1.5719      0.075    -20.842      0.000      -1.720      -1.424
C(countyFIPS)[T.8033.0]     -2.1078      0.202    -10.457      0.000      -2.503      -1.713
C(countyFIPS)[T.8035.0]     -2.1357      0.071    -30.153      0.000      -2.275      -1.997
C(countyFIPS)[T.8037.0]     -2.0837      0.180    -11.605      0.000      -2.436      -1.732
C(countyFIPS)[T.8039.0]     -0.0624      0.192     -0.325      0.746      -0.439       0.314
C(countyFIPS)[T.8041.0]      0.9486      0.081     11.742      0.000       0.790       1.107
C(countyFIPS)[T.8043.0]     -1.0836      0.182     -5.957      0.000      -1.440      -0.727
C(countyFIPS)[T.8045.0]     -1.1261      0.178     -6.339      0.000      -1.474      -0.778
C(countyFIPS)[T.8047.0]     -1.9526      0.200     -9.761      0.000      -2.345      -1.561
C(countyFIPS)[T.8049.0]     -1.4009      0.196     -7.143      0.000      -1.785      -1.016
C(countyFIPS)[T.8051.0]     -2.2930      0.196    -11.721      0.000      -2.676      -1.910
C(countyFIPS)[T.8053.0]     -1.6698      0.202     -8.262      0.000      -2.066      -1.274
C(countyFIPS)[T.8055.0]     -1.3792      0.200     -6.910      0.000      -1.770      -0.988
C(countyFIPS)[T.8057.0]     -1.9280      0.202     -9.551      0.000      -2.324      -1.532
C(countyFIPS)[T.8059.0]     -1.8116      0.037    -48.440      0.000      -1.885      -1.738
C(countyFIPS)[

In [14]:
#PPCSTOT
sp2 = """PPCSTOT ~ KPost2012 + pop10 + str + imtr + cmtr + +C(countyFIPS) + tt_CO +
tt_KS + tt_MO + tt_NE + tt_OK"""
PPCSTOT = smf.ols(formula=sp2, data=educ).fit(cov_type ='cluster', cov_kwds=
                                             {'groups': educ.stateFIPS})
PPCSTOT.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                PPCSTOT   R-squared:                       0.975
Model:                            OLS   Adj. R-squared:                  0.968
Method:                 Least Squares   F-statistic:                -1.337e+17
Date:                Tue, 01 May 2018   Prob (F-statistic):               1.00
Time:                        21:04:54   Log-Likelihood:                -22267.
No. Observations:                2260   AIC:                         4.546e+04
Df Residuals:                    1799   BIC:                         4.809e+04
Df Model:                         460                                         
Covariance Type:              cluster                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                 9.499e+04   2.54e+04      3.742      0.000    4.52e+04    1.45e+05
C(countyFIPS)[T.8003.0]  -3.981e+04   9256.370     -4.300      0.000   -5.79e+04   -2.17e+04
C(countyFIPS)[T.8005.0]   1.334e+04   2810.621      4.746      0.000    7830.951    1.88e+04
C(countyFIPS)[T.8007.0]   -4.94e+04   9340.118     -5.289      0.000   -6.77e+04   -3.11e+04
C(countyFIPS)[T.8009.0]   1.079e+04   9514.450      1.134      0.257   -7857.163    2.94e+04
C(countyFIPS)[T.8011.0]  -4.019e+04   9466.708     -4.245      0.000   -5.87e+04   -2.16e+04
C(countyFIPS)[T.8013.0]  -3.979e+04   3248.976    -12.245      0.000   -4.62e+04   -3.34e+04
C(countyFIPS)[T.8015.0]  -3.875e+04   9215.040     -4.205      0.000   -5.68e+04   -2.07e+04
C(countyFIPS)[T.8017.0]  -2.651e+04   9553.492     -2.775      0.006   -4.52e+04   -7782.988
C(countyFIPS)[T.8019.0]  -4.827e+04   9402.820     -5.133      0.000   -6.67e+04   -2.98e+04
C(countyFIPS)[T.8021.0]  -2.745e+04   9419.878     -2.914      0.004   -4.59e+04   -8991.145
C(countyFIPS)[T.8023.0]  -3.519e+04   9517.951     -3.697      0.000   -5.38e+04   -1.65e+04
C(countyFIPS)[T.8025.0]  -4.949e+04   9475.482     -5.223      0.000   -6.81e+04   -3.09e+04
C(countyFIPS)[T.8027.0]  -4.914e+04   9503.335     -5.170      0.000   -6.78e+04   -3.05e+04
C(countyFIPS)[T.8029.0]  -5.068e+04   8960.195     -5.656      0.000   -6.82e+04   -3.31e+04
C(countyFIPS)[T.8031.0]  -4.633e+04   3573.156    -12.966      0.000   -5.33e+04   -3.93e+04
C(countyFIPS)[T.8033.0]  -4.814e+04   9550.115     -5.041      0.000   -6.69e+04   -2.94e+04
C(countyFIPS)[T.8035.0]   -4.98e+04   3355.669    -14.841      0.000   -5.64e+04   -4.32e+04
C(countyFIPS)[T.8037.0]  -4.818e+04   8506.938     -5.664      0.000   -6.49e+04   -3.15e+04
C(countyFIPS)[T.8039.0]   2.165e+04   9103.303      2.378      0.017    3809.944    3.95e+04
C(countyFIPS)[T.8041.0]   6.917e+04   3827.232     18.072      0.000    6.17e+04    7.67e+04
C(countyFIPS)[T.8043.0]  -3.068e+04   8617.873     -3.560      0.000   -4.76e+04   -1.38e+04
C(countyFIPS)[T.8045.0]  -3.202e+04   8415.830     -3.805      0.000   -4.85e+04   -1.55e+04
C(countyFIPS)[T.8047.0]  -4.643e+04   9476.942     -4.900      0.000    -6.5e+04   -2.79e+04
C(countyFIPS)[T.8049.0]  -3.772e+04   9291.944     -4.059      0.000   -5.59e+04   -1.95e+04
C(countyFIPS)[T.8051.0]  -4.995e+04   9268.965     -5.389      0.000   -6.81e+04   -3.18e+04
C(countyFIPS)[T.8053.0]  -4.241e+04   9575.277     -4.429      0.000   -6.12e+04   -2.36e+04
C(countyFIPS)[T.8055.0]  -3.713e+04   9456.321     -3.927      0.000   -5.57e+04   -1.86e+04
C(countyFIPS)[T.8057.0]   -4.61e+04   9563.584     -4.820      0.000   -6.48e+04   -2.74e+04
C(countyFIPS)[T.8059.0]  -4.815e+04   1771.860    -27.172      0.000   -5.16e+04   -4.47e+04
C(countyFIPS)[

In [15]:
#TCURSPND
sp3 = """np.log(educ.TCURSPND) ~ KPost2012 + pop10 + str + imtr + cmtr + +C(countyFIPS) + tt_CO +
tt_KS + tt_MO + tt_NE + tt_OK"""
TCURSPNDlog = smf.ols(formula=sp3, data=educ).fit(cov_type ='cluster', cov_kwds=
                                             {'groups': educ.stateFIPS})
TCURSPNDlog.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     np.log(educ.TCURSPND)   R-squared:                       0.999
Model:                               OLS   Adj. R-squared:                  0.999
Method:                    Least Squares   F-statistic:                -2.182e+18
Date:                   Tue, 01 May 2018   Prob (F-statistic):               1.00
Time:                           21:04:56   Log-Likelihood:                 4027.1
No. Observations:                   2260   AIC:                            -7132.
Df Residuals:                       1799   BIC:                            -4494.
Df Model:                            460                                         
Covariance Type:                 cluster                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   17.2744      0.901     19.175      0.000      15.509      19.040
C(countyFIPS)[T.8003.0]     -2.1712      0.321     -6.756      0.000      -2.801      -1.541
C(countyFIPS)[T.8005.0]      0.0073      0.098      0.075      0.940      -0.184       0.199
C(countyFIPS)[T.8007.0]     -2.8361      0.324     -8.745      0.000      -3.472      -2.200
C(countyFIPS)[T.8009.0]     -3.1394      0.330     -9.503      0.000      -3.787      -2.492
C(countyFIPS)[T.8011.0]     -3.3814      0.329    -10.288      0.000      -4.026      -2.737
C(countyFIPS)[T.8013.0]      0.1229      0.113      1.089      0.276      -0.098       0.344
C(countyFIPS)[T.8015.0]     -2.3734      0.320     -7.418      0.000      -3.001      -1.746
C(countyFIPS)[T.8017.0]     -3.8144      0.332    -11.499      0.000      -4.465      -3.164
C(countyFIPS)[T.8019.0]     -3.0712      0.326     -9.407      0.000      -3.711      -2.431
C(countyFIPS)[T.8021.0]     -2.6488      0.327     -8.099      0.000      -3.290      -2.008
C(countyFIPS)[T.8023.0]     -3.6364      0.330    -11.004      0.000      -4.284      -2.989
C(countyFIPS)[T.8025.0]     -3.9358      0.329    -11.963      0.000      -4.581      -3.291
C(countyFIPS)[T.8027.0]     -3.9553      0.330    -11.987      0.000      -4.602      -3.309
C(countyFIPS)[T.8029.0]     -1.7009      0.311     -5.467      0.000      -2.311      -1.091
C(countyFIPS)[T.8031.0]     -0.2518      0.124     -2.029      0.042      -0.495      -0.009
C(countyFIPS)[T.8033.0]     -4.2525      0.332    -12.825      0.000      -4.902      -3.603
C(countyFIPS)[T.8035.0]      0.0483      0.117      0.414      0.679      -0.180       0.277
C(countyFIPS)[T.8037.0]     -1.3269      0.295     -4.492      0.000      -1.906      -0.748
C(countyFIPS)[T.8039.0]     -2.0254      0.316     -6.408      0.000      -2.645      -1.406
C(countyFIPS)[T.8041.0]     -0.2263      0.133     -1.703      0.089      -0.487       0.034
C(countyFIPS)[T.8043.0]     -1.6871      0.299     -5.638      0.000      -2.274      -1.101
C(countyFIPS)[T.8045.0]     -0.9320      0.292     -3.190      0.001      -1.505      -0.359
C(countyFIPS)[T.8047.0]     -3.8457      0.329    -11.688      0.000      -4.491      -3.201
C(countyFIPS)[T.8049.0]     -2.5329      0.323     -7.851      0.000      -3.165      -1.901
C(countyFIPS)[T.8051.0]     -2.6418      0.322     -8.209      0.000      -3.273      -2.011
C(countyFIPS)[T.8053.0]     -5.0153      0.332    -15.086      0.000      -5.667      -4.364
C(countyFIPS)[T.8055.0]     -3.3088      0.328    -10.078      0.000      -3.952      -2.665
C(countyFIPS)[T.8057.0]     -4.4042      0.332    -13.264      0.000      -5.055      -3.753
C(countyFIPS)[T.8059.0]     -0.1775      0.062     -2.886      0.004      

In [16]:
#TCURSPND
sp3 = """educ.TCURSPND ~ KPost2012 + pop10 + str + imtr + cmtr + +C(countyFIPS) + tt_CO +
tt_KS + tt_MO + tt_NE + tt_OK"""
TCURSPND = smf.ols(formula=sp3, data=educ).fit(cov_type ='cluster', cov_kwds=
                                             {'groups': educ.stateFIPS})
TCURSPND.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          educ.TCURSPND   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.392e+15
Date:                Tue, 01 May 2018   Prob (F-statistic):           1.04e-30
Time:                        21:04:57   Log-Likelihood:                -22174.
No. Observations:                2260   AIC:                         4.527e+04
Df Residuals:                    1799   BIC:                         4.791e+04
Df Model:                         460                                         
Covariance Type:              cluster                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                -4.004e+05   3.72e+05     -1.076      0.282   -1.13e+06    3.29e+05
C(countyFIPS)[T.8003.0]   1.463e+05   1.33e+05      1.097      0.273   -1.15e+05    4.08e+05
C(countyFIPS)[T.8005.0]   5.706e+04   4.05e+04      1.409      0.159   -2.23e+04    1.36e+05
C(countyFIPS)[T.8007.0]   1.414e+05   1.35e+05      1.051      0.293   -1.22e+05    4.05e+05
C(countyFIPS)[T.8009.0]   1.535e+05   1.37e+05      1.120      0.263   -1.15e+05    4.22e+05
C(countyFIPS)[T.8011.0]   1.473e+05   1.36e+05      1.080      0.280    -1.2e+05    4.15e+05
C(countyFIPS)[T.8013.0]   1.156e+05   4.68e+04      2.470      0.014    2.39e+04    2.07e+05
C(countyFIPS)[T.8015.0]   1.381e+05   1.33e+05      1.040      0.298   -1.22e+05    3.98e+05
C(countyFIPS)[T.8017.0]   1.524e+05   1.38e+05      1.107      0.268   -1.17e+05    4.22e+05
C(countyFIPS)[T.8019.0]   1.443e+05   1.35e+05      1.065      0.287   -1.21e+05     4.1e+05
C(countyFIPS)[T.8021.0]   1.511e+05   1.36e+05      1.113      0.266   -1.15e+05    4.17e+05
C(countyFIPS)[T.8023.0]   1.502e+05   1.37e+05      1.095      0.273   -1.19e+05    4.19e+05
C(countyFIPS)[T.8025.0]   1.449e+05   1.37e+05      1.061      0.288   -1.23e+05    4.13e+05
C(countyFIPS)[T.8027.0]   1.473e+05   1.37e+05      1.076      0.282   -1.21e+05    4.16e+05
C(countyFIPS)[T.8029.0]   1.364e+05   1.29e+05      1.056      0.291   -1.17e+05    3.89e+05
C(countyFIPS)[T.8031.0]  -1.642e+05   5.15e+04     -3.189      0.001   -2.65e+05   -6.33e+04
C(countyFIPS)[T.8033.0]   1.505e+05   1.38e+05      1.093      0.274   -1.19e+05     4.2e+05
C(countyFIPS)[T.8035.0]   8.045e+04   4.84e+04      1.664      0.096   -1.43e+04    1.75e+05
C(countyFIPS)[T.8037.0]   1.183e+05   1.23e+05      0.965      0.335   -1.22e+05    3.59e+05
C(countyFIPS)[T.8039.0]   1.371e+05   1.31e+05      1.045      0.296    -1.2e+05    3.94e+05
C(countyFIPS)[T.8041.0]  -1.367e+05   5.51e+04     -2.479      0.013   -2.45e+05   -2.86e+04
C(countyFIPS)[T.8043.0]   1.081e+05   1.24e+05      0.870      0.384   -1.35e+05    3.51e+05
C(countyFIPS)[T.8045.0]   1.425e+05   1.21e+05      1.175      0.240   -9.51e+04     3.8e+05
C(countyFIPS)[T.8047.0]   1.455e+05   1.37e+05      1.065      0.287   -1.22e+05    4.13e+05
C(countyFIPS)[T.8049.0]   1.417e+05   1.34e+05      1.059      0.290   -1.21e+05    4.04e+05
C(countyFIPS)[T.8051.0]   1.379e+05   1.34e+05      1.033      0.302   -1.24e+05       4e+05
C(countyFIPS)[T.8053.0]   1.511e+05   1.38e+05      1.095      0.273   -1.19e+05    4.22e+05
C(countyFIPS)[T.8055.0]   1.469e+05   1.36e+05      1.078      0.281    -1.2e+05    4.14e+05
C(countyFIPS)[T.8057.0]   1.512e+05   1.38e+05      1.098      0.272   -1.19e+05    4.21e+05
C(countyFIPS)[T.8059.0]  -1.262e+05   2.55e+04     -4.942      0.000   -1.76e+05   -7.61e+04
C(countyFIPS)[

In [17]:
from statsmodels.iolib.summary2 import summary_col
output = summary_col([TotalEXPlog,TotalEXP,PPCSTOT,PPCSTOTlog,
                      TCURSPNDlog, TCURSPND],stars=True)

In [18]:
output

,np.log(TOTALEXP),TOTALEXP,PPCSTOT,np.log(PPCSTOT),np.log(educ.TCURSPND),educ.TCURSPND
Intercept,17.1529***,-199684.5737,94993.6581***,13.0379***,17.2744***,-400394.1777
,(0.7625),(394675.8442),(25384.2015),(0.5281),(0.9009),(372039.6831)
C(countyFIPS)[T.8003.0],-1.7444***,89759.2848,-39805.0751***,-1.5057***,-2.1712***,146315.2845
,(0.2761),(137928.6968),(9256.3696),(0.1954),(0.3214),(133373.6828)
C(countyFIPS)[T.8005.0],-0.0168,121191.1767***,13339.6681***,0.3212***,0.0073,57055.7773
,(0.0838),(41880.9264),(2810.6213),(0.0593),(0.0976),(40497.8334)
C(countyFIPS)[T.8007.0],-2.7297***,71248.6188,-49397.9117***,-2.2335***,-2.8361***,141420.3370
,(0.2786),(139176.6208),(9340.1176),(0.1971),(0.3243),(134580.3949)
C(countyFIPS)[T.8009.0],-3.0529***,83488.1965,10790.8161,-0.2081,-3.1394***,153529.6075
,(0.2838),(141774.3353),(9514.4497),(0.2008),(0.3303),(137092.3214)
